# Visualizing 3D data in napari
In this tutorial, you will learn:
- How to load images from hdf5 files
- How to open images in napari
- The difference between intensity images and label images
- How to visualize measurements on images using a napari plugin
- How to explore image data interactively

In [ ]:
import h5py
# We import some custom functions to handle the hdf5 files
from h5_files import h5_select, h5_summary, to_numpy

In [ ]:
# Get an overview of what's in an hdf5 file
file_path = 'PATH/TO/H5/FILE'
print(h5_summary(file_path))

In [ ]:
# Load in the images from the hd5f file
f = h5py.File(file_path, "r")
sytox = h5_select(f, {'stain': 'SYTOX', 'level': 0})[0]
sytox_np = to_numpy(sytox)

In [ ]:
# TODO: Check what we loaded with the h5_select function

In [ ]:
# TODO: Load the PCNA, the bCatenin and the 5-EU images at level 0

In [ ]:
# Loading the segmentation mask for the nuclei
nuclei = h5_select(f, {'stain': 'nuclei', 'level': 2})[0]
nuclei_np = to_numpy(nuclei)

In [ ]:
# TODO: Load the segmentation masks for the cells & the membranes

In [ ]:
# Each dataset contains the actual data plus some metadata attributes
# As you can see here, the nuclear segmentation was done using cellpose: https://www.cellpose.org 
list(nuclei.attrs)

## The napari viewer
napari is a fast, interactive, multi-dimensional image viewer for Python. It’s designed for browsing, annotating, and analyzing large multi-dimensional images.  
https://napari.org

In [ ]:
import napari

In [ ]:
# Open the image and the segmentation in the napari viewer
viewer = napari.view_image(sytox_np, scale=sytox.attrs['element_size_um'])
viewer.add_labels(nuclei_np, scale=nuclei.attrs['element_size_um'])

In [ ]:
# TODO: Add other intensity images & segmentations to the viewer & explore
# Make sure to add the membrane segmentation to see the cell boundaries

# Questions to look at during the exploration
- How does nuclear size & cell size vary over the embryo?
- How does the 5-EU signal vary over the different nuclei and what does that mean?
- How does PCNA vary over the embryo and what does that mean?
- What would you need to measure to quantitatively answer those questions?
- What potential technical biases do you see in the images that we should be aware of when quantifying them?

## Visualize feature measurements
Using the napari feature visualization plugin, look at some of the features, e.g.:  
- Physical Size of the nuclei or the cell. Roundness
- EU intensity (mean, skewness)
- PCNA intensities (mean)  
Do the quantifications help you answer the questions above?